# Custom Model Monitoring on Verta

Verta provides a extensible [model monitoring framework](https://docs.verta.ai/verta/monitoring) that allows the user to fully define and configure what data to monitor and how to monitor it including model input and output.

This notebook shows an example of how Verta model monitoring can be used to define custom monitors on monitor I/O of a census prediction model.

## 0. Imports

In [ ]:
from __future__ import print_function

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

### 0.1 Verta import and setup

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [ ]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST']

from verta import Client
client = Client(os.environ['VERTA_HOST'])

## 1. Fetch data

In [ ]:
try:
    import wget
except ImportError:
    !pip install wget  # you may need pip3
    import wget

In [ ]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(train_data_url)

In [ ]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

In [ ]:
df_train.head()

## 2. Define monitored entities

In Verta Model Monitoring, a Monitored entity (ME) encapsulates the thing being monitored, e.g., a model, a pipeline, and acts as a context within which data summaries are produced and analyzed

In [ ]:
me = client.monitoring.get_or_create_monitored_entity("census-income-model")

## 2.1 Define data summaries and summary samples

For a specific ME, there are particular aspects of the data that we wish to monitor, e.g., for a model, we may want to monitor the inputs and outputs; for a dataset, we may want to monitor values in each column of the dataset.

So the next step is to define the data _summaries_ we wish to capture. Within Verta's tools for monitoring, a summary defines a class of data statistics which the user is interested in, for example a mean squared error or a histogram of data table column values. A summary sample is an instance of that summary, which might be logged from a training epoch or a batch of inputs and outputs for a deployed model.

In [ ]:
# Suppose in this case, we would like to monitor data summaries for a specific set of columns in our data, then
# here's how we could define a generic function to define those summaries

continuous_columns = ["age", "capital-gain", "capital-loss","hours-per-week"]
discrete_columns = ["workclass_local-gov", "workclass_private", "workclass_self-emp-inc", "workclass_self-emp-not-inc", "workclass_state-gov","workclass_without-pay",
                    "education_11th","education_12th","education_1st-4th","education_5th-6th","education_7th-8th","education_9th","education_assoc-acdm","education_assoc-voc","education_bachelors","education_doctorate","education_hs-grad","education_masters","education_preschool","education_prof-school","education_some-college",
                   "relationship_not-in-family","relationship_other-relative","relationship_own-child","relationship_unmarried","relationship_wife",
                    "occupation_armed-forces","occupation_craft-repair","occupation_exec-managerial","occupation_farming-fishing","occupation_handlers-cleaners","occupation_machine-op-inspct","occupation_other-service","occupation_priv-house-serv","occupation_prof-specialty","occupation_protective-serv","occupation_sales","occupation_tech-support","occupation_transport-moving",
                   ">50k"]
all_columns = continuous_columns + discrete_columns

from verta.data_types import (
    DiscreteHistogram,
    FloatHistogram,
    NumericValue,
)

from verta.monitoring.profiler import (
    MissingValuesProfiler,
    BinaryHistogramProfiler,
    ContinuousHistogramProfiler,
)

def profile(data, labels, start_time, end_time, monitored_entity):        
    bin_ranges = {}
    for col in continuous_columns:
        bin_ranges[col] = (10, 10) if col in ["age", "hours-per-week"] else (500, 20)
    for col in continuous_columns:
        summary_name = col + "-Histogram"
        summary = client.monitoring.summaries.get_or_create(summary_name, FloatHistogram, monitored_entity)
        summary_samples = ContinuousHistogramProfiler(columns=[col], bins=[x*bin_ranges[col][0] for x in range(bin_ranges[col][1])]).profile(data)

        for _, histogram in summary_samples.items():  
            summary.log_sample(histogram, labels, start_time, end_time)
        
    for col in discrete_columns:    
        summary_name = col + "-Histogram"
        summary = client.monitoring.summaries.get_or_create(summary_name, DiscreteHistogram, monitored_entity)
        summary_samples = BinaryHistogramProfiler(columns=[col]).profile(data)

        for _, histogram in summary_samples.items():  
            summary.log_sample(histogram, labels, start_time, end_time)

    for col in all_columns:
        missing_summary_name = col + "-Missing"
        missing_summary = client.monitoring.summaries.get_or_create(missing_summary_name, DiscreteHistogram, monitored_entity)                
        summary_samples = MissingValuesProfiler(columns=[col]).profile(data)

        for _, missing_counts in summary_samples.items():  
            missing_summary.log_sample(missing_counts, labels, start_time, end_time)

## 2.2 Define alerts

In many ways, monitors and summaries are just a way to get to our objective; know when unexpected things happen in the system. So next, we define alerts to notify us when somethin unexpected happens

In [ ]:
from verta.monitoring.notification_channel import SlackNotificationChannel
from verta.monitoring.alert import ReferenceAlerter
from verta.monitoring.comparison import GreaterThan
from verta.monitoring.summaries.queries import SummaryQuery
from verta.monitoring.summaries.queries import SummarySampleQuery

In [ ]:
from datetime import datetime, timedelta, timezone

today = datetime.now(timezone.utc)

In [ ]:
channel = None

# supply a Slack notification channel, if available
# channel = monitoring.notification_channels.get_or_create(
#     "Demo Monitoring Alerts",
#     SlackNotificationChannel(webhook_url)
# )

def set_alerts(monitored_entity):
    summaries = client.monitoring.summaries.find(SummaryQuery(
            monitored_entities=[monitored_entity.id],
        ))
    for summary in summaries:
        threshold = 0.2
        ref_sample = summary.find_samples(SummarySampleQuery(labels={"source":"reference"}))[0]
        alerter = ReferenceAlerter(
            GreaterThan(threshold),
            ref_sample,
        )
        alert = summary.alerts.create(
            summary.name + "- ReferenceDeviation GT {}".format(threshold),
            alerter,
            # notification_channels=[channel], # uncomment if channel is supplied
            starting_from=today-timedelta(hours=30), # pick a suitable time from which the alerter should be enabled
        )

## 3. Incorporate profiling functions into your workflow
A typical data monitoring workflow works as follows: 
1. Log reference summaries (e.g., for training data, at training time)
2. Log live/new summaries (e.g., when a daily job is re-run or when a model makes predictions)

### 3.1 Log reference summaries

In [ ]:
profile(df_train, {"source" : "reference"}, today - timedelta(hours=120), today - timedelta(hours=120), me)

In [ ]:
# note: as defined above, our alerts need a reference sample to work correctly, so alerts must be set after logging
# reference summary samples
set_alerts(me)

### 3.2 Log live/new summaries

Suppose in this case that we have a new dataset and we want to make sure that the new data matches the reference one.

### Log data that looks like the reference and should not produce alerts

In [ ]:
test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(test_data_url)

In [ ]:
df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:, -1]

In [ ]:
df_test.head()

In [ ]:
profile(
    df_test, 
    {"source" : "test-data"}, 
    today-timedelta(hours=90), 
    today-timedelta(hours=90),
    me
    )

### Log data with drift (in age) that does not look like the reference and should produce alerts

In [ ]:
test_drift_data_url = "http://s3.amazonaws.com/verta-starter/census-test-age-drift.csv"
test_drift_data_filename = wget.detect_filename(test_drift_data_url)
if not os.path.isfile(test_drift_data_filename):
    wget.download(test_drift_data_url)

In [ ]:
#df_test_drift = pd.read_csv(test_drift_data_filename)
df_test_drift = pd.read_csv("census-test-age-drift.csv")
X_test_drift = df_test_drift.iloc[:,:-1]
y_test_drift = df_test_drift.iloc[:, -1]

In [ ]:
df_test_drift.head()

In [ ]:
profile(
    df_test_drift, 
    {"source" : "age-drift"}, 
    today-timedelta(hours=30), 
    today-timedelta(hours=30),
    me
    )